In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from scipy import integrate
import math

In [2]:
def F(y,x,Sn,Sl):
    beta = (Sn/Sl)**1.5
    f = 1#2**0.5
    num = (x**2)*(y**2)*(np.exp(-x))*(np.exp(-y))  #
    deno = (np.exp(-x)/(beta*f)) + (np.exp(-y)/f)
    return num/deno
    

In [46]:
filename = '/Users/rajorshi/Desktop/Na_wannier.xyz'
filename_spread = '/Users/rajorshi/Desktop/Na_spread.xyz'

with open(filename) as f:
    lines = f.readlines()
with open(filename_spread) as f_sp:
    lines_sp = f_sp.readlines()

wan_cent = {}
wan_sp = {}
count = 0
count_C = 0
count_sp = 2

for i in range(len(lines)):
    s = lines[i].split()
    if s[0]=='X':
        count = count+1
        if count == 1:
            start_wan = i

for i in range(len(lines)):
    s = lines[i].split()
    if s[0]=='Na':
        count_C = count_C + 1
        key = 'Na_'+str(count_C)
        
        temp = lines[start_wan].split()
        temp_1 = lines[start_wan+1].split()
        temp_2 = lines[start_wan+2].split()
        temp_3 = lines[start_wan+3].split()
        temp_sp = float(lines_sp[count_sp].split()[2])
        temp_sp_1 = float(lines_sp[count_sp+1].split()[2])
        temp_sp_2 = float(lines_sp[count_sp+2].split()[2])
        temp_sp_3 = float(lines_sp[count_sp+3].split()[2])
        
        wan_1 = [float(temp[1]),float(temp[2]),float(temp[3])]
        wan_2 = [float(temp_1[1]),float(temp_1[2]),float(temp_1[3])]
        wan_3 = [float(temp_2[1]),float(temp_2[2]),float(temp_2[3])]
        wan_4 = [float(temp_3[1]),float(temp_3[2]),float(temp_3[3])]
        
        wan_cent.update({key:(wan_1,wan_2,wan_3,wan_4)})
        wan_sp.update({key:(temp_sp,temp_sp_1,temp_sp_2,temp_sp_3)})
        
        start_wan = start_wan + 4
        count_sp = count_sp + 4
        

In [47]:
wan_cent

{'Na_1': ([0.1232538046, 0.1232533043, 0.123254142],
  [0.0396670162, -0.2060166415, 0.0396669635],
  [-0.206016711, 0.0396676333, 0.039667345],
  [0.0396675762, 0.0396677848, -0.206016418]),
 'Na_2': ([0.1232538046, 0.1232533043, 0.123254142],
  [0.0396670162, -0.2060166415, 0.0396669635],
  [-0.206016711, 0.0396676333, 0.039667345],
  [0.0396675762, 0.0396677848, -0.206016418])}

In [3]:
from scipy import integrate

conv = 1#1.88973
inte = []

for spe in wan_cent.keys():
    key_1 = spe
    for j in range(len(wan_cent[key_1])):
        wan_1 = wan_cent[key_1][j]
        sp_1 = ((wan_sp[key_1][j])**0.5)/(2*math.pi)
        #print(key_1,wan_1,sp_1)
        for key_2 in wan_cent.keys():
            if key_2 != key_1:
                for p in range(len(wan_cent[key_2])): 
                    wan_2 = wan_cent[key_2][p]
                    sp_2 = ((wan_sp[key_2][p])**0.5)/(2*math.pi)
                    #print(key_2,wan_2,sp_2)
                    rc_1 = sp_1*conv*(3**0.5)*(0.769+(0.5*np.log(sp_1*conv)))
                    rc_2 = sp_2*conv*(3**0.5)*(0.769+(0.5*np.log(sp_2*conv)))
                    rc_1_1 = sp_1*conv*(1.475-(0.866*np.log(sp_1*conv)))
                    rc_2_2 = sp_2*conv*(1.475-(0.866*np.log(sp_2*conv)))
                    lim_x = ((3**0.5)*rc_1)/(sp_1*conv)
                    lim_y = ((3**0.5)*rc_2)/(sp_2*conv)
                    print(rc_1,rc_2)
                    #print(rc_1_1,rc_2_2)
                    res = integrate.dblquad(F, 0, lim_x, 0, lim_y, args = (sp_1,sp_2))
                    print(res)
                    prefac = ((sp_1**1.5)*(sp_2**3)*(conv**4.5))/(2*(3**1.25))
                    #print(prefac)
                    inte.append(prefac*res[0])

NameError: name 'wan_cent' is not defined

In [120]:
sp_1 =(3**0.5)
sp_2 =(3**0.5)
conv =1
rc_1 = (sp_1*conv*(3**0.5))*(0.769+(0.5*np.log(sp_1*conv)))
rc_2 = (sp_2*conv)*(3**0.5)*(0.769+(0.5*np.log(sp_2*conv)))
#rc_1_1 = sp_1*conv*(1.475-(0.866*np.log(sp_1*conv)))
#rc_2_2 = sp_2*conv*(1.475-(0.866*np.log(sp_2*conv)))
lim_x = ((3**0.5)*rc_1)/(sp_1*conv)
lim_y = ((3**0.5)*rc_2)/(sp_2*conv)
print(rc_1,rc_2)
print(rc_1_1,rc_2_2)
#res = integrate.dblquad(F, 0, lim_x, 0, lim_y, args = (sp_1,sp_2))
res = integrate.nquad(F, [[0,lim_y],[0,lim_x]], args = (sp_1,sp_2))
print(res)
prefac = ((sp_1**1.5)*(sp_2**3)*(conv**4.5))/(2*(3**1.25))
print(prefac,prefac*res[0])

3.1309592165010818 3.1309592165010818
1.6923927060099657 1.6923927060099657
(5.012237398136023, 5.5647013642979e-14)
1.4999999999999996 7.518356097204031


In [104]:
6.51/prefac

4.340000000000001

In [42]:
sum(inte[0:16])

0.5941481103688326

In [43]:
(sum(inte)/len(inte))*(2**1.5)

0.10503153946774371

In [88]:
#empirical relation

Za = 1
Zb = 1
Sa = 1.5502489404291333#3**0.5
Sb = 1.5502489404291333#3**0.5
pol_H = 4.5
gam = pol_H/(3**1.5)

C6 = (1.5*((Za*Zb)**0.5)*(Sa*Sb)**3*(gam**1.5))/(Zb**0.5*Sa**1.5 + Za**0.5*Sb**1.5)
C6*2

8.693480832770453

In [32]:
wan_cent

{'Na_1': ([0.1232538046, 0.1232533043, 0.123254142],
  [0.0396670162, -0.2060166415, 0.0396669635],
  [-0.206016711, 0.0396676333, 0.039667345],
  [0.0396675762, 0.0396677848, -0.206016418]),
 'Na_2': ([0.1232538046, 0.1232533043, 0.123254142],
  [0.0396670162, -0.2060166415, 0.0396669635],
  [-0.206016711, 0.0396676333, 0.039667345],
  [0.0396675762, 0.0396677848, -0.206016418])}

In [83]:
((0.67298188)**0.5)*1.88973

1.5502489404291333

In [103]:
np.log(10)

2.302585092994046

In [3]:
def calc_coeff(wan_frag_1,wan_frag_2,wan_frag_1_dist,wan_frag_2_dist):
    conv = 1#1.88973
    conv_1 = 1.88973  #ang to bohr
    inte = []
    C6 = []
    C8 = []
    
    for ele1,ele1_dist in zip(wan_frag_1,wan_frag_1_dist):  #extract spread and distance for elements from frag 1
        count = 0
        for ele2,ele2_dist in zip(wan_frag_2,wan_frag_2_dist): #extract spread and distance for elements from frag 2
            temp_6 = 0
            temp_8 = 0
            if (ele1 != ele2).all():  #remove self interactions
                for wan1,dist1 in zip(ele1,ele1_dist):    #iterate over wannier centers of both fragments
                    for wan2,dist2 in zip(ele2,ele2_dist): 
                        sp_1 = (wan1)**0.5                #sqaure root of spread (in bohr)
                        sp_2 = (wan2)**0.5
                        #print(sp_1,sp_2)
                        rc_1 = sp_1*conv*(3**0.5)*(0.769+(0.5*np.log(sp_1*conv)))
                        rc_2 = sp_2*conv*(3**0.5)*(0.769+(0.5*np.log(sp_2*conv)))
                        lim_x = ((3**0.5)*rc_1)/(sp_1*conv)  #limits of integration
                        lim_y = ((3**0.5)*rc_2)/(sp_2*conv)
        
                        res = integrate.dblquad(F, 0, lim_x, 0, lim_y, args = (sp_1,sp_2))
                        prefac = ((sp_1**1.5)*(sp_2**3)*(conv**4.5))/(2*(3**1.25))
                        temp_6 = temp_6 + (prefac*res[0])     #calculate C6 for current wannier pair
                        temp_8 = temp_8 + ((prefac*res[0])*5*(conv_1**2)*((dist1**2)+(dist2**2)))  #calculate C8 from C6
                C6.append(temp_6)
                C8.append(temp_8)
            else:
                continue
            
    return C6,C8    

In [4]:
def calc_dist(ion,wan):
    #print(ion,wan)
    d = []
    x = float(ion.split()[1])
    y = float(ion.split()[2])
    z = float(ion.split()[3])
    for i in range(len(wan)):
        wan_x = float(wan[i].split()[1])
        wan_y = float(wan[i].split()[2])
        wan_z = float(wan[i].split()[3])
        dist = ((wan_x-x)**2 + (wan_y-y)**2 + (wan_z-z)**2)**0.5  #calculate distance from ion to wannier
        d.append(dist)
    return d

In [5]:
#driver code for calculating C6 and C8 for unlike fragments (eg. Na-Cl,Cl-O...)

frag = 2                #enter number of fragments
n_frag_1 = 32           #number of atoms in fragment 1
n_frag_2 = 96           #number of atoms in fragment 2
n_wan_at = 4            #number of wannier centers associated with each atom


#filename_wan_sort = '/Users/rajorshi/Desktop/AIMD/773K_5kbar/Cl-O/110_frame_conv/wannier_sort.xyz' #wannier centers sorted according to ions
#filename_sp_sort = '/Users/rajorshi/Desktop/AIMD/773K_5kbar/Cl-O/110_frame_conv/spread_sort.xyz' #wannier spreads sorted accoridng to ions
#filename_uw = '/Users/rajorshi/Desktop/AIMD/773K_5kbar/Cl-O/110_frame_conv/wannier_sort_uw.xyz' #unwrapped coordinates

filename_wan_sort = '/Users/rajorshi/Desktop/AIMD/LaCl3_crystal/15_frame_conv_hyd/wannier_sort.xyz' #wannier centers sorted according to ions
filename_sp_sort = '/Users/rajorshi/Desktop/AIMD/LaCl3_crystal/15_frame_conv_hyd/spread_sort.xyz' #wannier spreads sorted accoridng to ions
filename_uw = '/Users/rajorshi/Desktop/AIMD/LaCl3_crystal/15_frame_conv_hyd/wannier_sort_uw.xyz' #unwrapped coordinate

#filename_wan_sort = '/Users/rajorshi/Desktop/AIMD/Traj_yuan_mei/773K_5kbar_LaCl3/50_frame_conv/wannier_sort.xyz' #wannier centers sorted according to ions
#filename_sp_sort = '/Users/rajorshi/Desktop/AIMD/Traj_yuan_mei/773K_5kbar_LaCl3/50_frame_conv/spread_sort.xyz' #wannier spreads sorted accoridng to ions
#filename_uw = '/Users/rajorshi/Desktop/AIMD/Traj_yuan_mei/773K_5kbar_LaCl3/50_frame_conv/wannier_sort_uw.xyz' #unwrapped coordinates

with open(filename_wan_sort) as f:
    lines = f.readlines()
with open(filename_sp_sort) as f1:
    lines_sp = f1.readlines()
with open(filename_uw) as f2:
    lines_uw = f2.readlines()

n_wan = int(lines_sp[0].split()[0])  #total number od wannier centers
n_atom = int(lines[0].split()[0])-n_wan #total number of atoms
start_sp = 2
temp_C6 = []
count_fr = 0
C6_final = []
C8_final = []

for i in range(len(lines_uw)):
    s = lines_uw[i].split()
    if len(s)==1:
        count_fr = count_fr + 1
        wan_frag_1 = np.zeros((n_frag_1,n_wan_at)) #2D array with rows as ions and columns are spreads(in bohr**2)
        wan_frag_2 = np.zeros((n_frag_2,n_wan_at))
        wan_frag_1_dist = np.zeros((n_frag_1,n_wan_at)) #2D array with rows as ions and columns are distance of wannier from ion(in ang)
        wan_frag_2_dist = np.zeros((n_frag_2,n_wan_at))
        temp_sp = lines_sp[start_sp:start_sp+n_wan]  #extract spreads for current frame
        start_sp = start_sp+n_wan+2
        coord = lines_uw[i+2:i+2+n_atom]   #extract unwrapped ion coordinates for current frame
        count_1 = 0
        count_2 = 0
        start = 0
        start_wan = i+2+n_atom
        print('Calculating {0} frame'.format(count_fr))
        for j in range(len(coord)):
            s1 = coord[j].split()
            
            if s1[0] == 'La':  #element in frag 1
                wan_1 = float(temp_sp[start].split()[2])   #extract wannier spreads
                wan_2 = float(temp_sp[start+1].split()[2])
                wan_3 = float(temp_sp[start+2].split()[2])
                wan_4 = float(temp_sp[start+3].split()[2])
                start = start + 4
                dist = calc_dist(coord[j],lines_uw[start_wan:start_wan+4]) #calculate distance of wannier from ion
                start_wan = start_wan + 4
                
                wan_frag_1[count_1][:] = wan_1,wan_2,wan_3,wan_4  
                wan_frag_1_dist[count_1][:] = dist[0],dist[1],dist[2],dist[3]
                count_1 = count_1 + 1
            elif s1[0] == 'Cl':  #element in frag 2
                wan_1 = float(temp_sp[start].split()[2])
                wan_2 = float(temp_sp[start+1].split()[2])
                wan_3 = float(temp_sp[start+2].split()[2])
                wan_4 = float(temp_sp[start+3].split()[2])
                start = start + 4
                dist = calc_dist(coord[j],lines_uw[start_wan:start_wan+4])
                start_wan = start_wan + 4
                
                wan_frag_2[count_2][:] = wan_1,wan_2,wan_3,wan_4
                wan_frag_2_dist[count_2][:] = dist[0],dist[1],dist[2],dist[3]
                count_2 = count_2 + 1
            elif s1[0] == 'O':
                start = start + 4
                start_wan = start_wan + 4
                
        #wan_frag_1 = wan_sp[0][:][:]
        #wan_frag_2 = wan_sp[1][:][:]
        #wan_frag_1_dist = wan_dist[0][:][:]
        #wan_frag_2_dist = wan_dist[1][:][:]
        #print(wan_frag_1_dist)
        #print(coord[-1])
        temp_C6,temp_C8=calc_coeff(wan_frag_1,wan_frag_2,wan_frag_1_dist,wan_frag_2_dist) #cal C6 and C8 for current frame
        #print(temp_C6,temp_C8)
        C6_final.extend(temp_C6)
        C8_final.extend(temp_C8)
        

Calculating 1 frame
Calculating 2 frame
Calculating 3 frame
Calculating 4 frame
Calculating 5 frame
Calculating 6 frame
Calculating 7 frame
Calculating 8 frame
Calculating 9 frame
Calculating 10 frame
Calculating 11 frame
Calculating 12 frame
Calculating 13 frame
Calculating 14 frame


In [22]:
wan_frag_1

array([[2.71829479, 2.69175644, 2.68023003, 2.67328609]])

In [8]:
#driver code for calculating C6 and C8 for like fragments (eg. Cl-Cl,Na-Na...)

frag = 2
n_frag_1 = 32
n_frag_2 = 32
n_wan_at = 4


filename_wan_sort = '/Users/rajorshi/Desktop/AIMD/LaCl3_crystal/15_frame_conv_hyd/wannier_sort.xyz'#'/Users/rajorshi/Desktop/AIMD/773K_1kbar/wan_2disp/test/wannier_sort.xyz'
filename_sp_sort = '/Users/rajorshi/Desktop/AIMD/LaCl3_crystal/15_frame_conv_hyd/spread_sort.xyz'
filename_sp_uw = '/Users/rajorshi/Desktop/AIMD/LaCl3_crystal/15_frame_conv_hyd/wannier_sort_uw.xyz'

with open(filename_wan_sort) as f:
    lines = f.readlines()
with open(filename_sp_sort) as f1:
    lines_sp = f1.readlines()
with open(filename_sp_uw) as f2:
    lines_uw = f2.readlines()

n_wan = int(lines_sp[0].split()[0])
n_atom = int(lines[0].split()[0])-n_wan
start_sp = 2
temp_C6 = []
count_fr = 0
C6_final = []
C8_final = []

for i in range(len(lines_uw)):
    s = lines_uw[i].split()
    if len(s)==1:
        count_fr = count_fr + 1
        wan_frag_1 = np.zeros((n_frag_1,n_wan_at))
        wan_frag_2 = np.zeros((n_frag_2,n_wan_at))
        wan_frag_1_dist = np.zeros((n_frag_1,n_wan_at))
        wan_frag_2_dist = np.zeros((n_frag_2,n_wan_at))
        temp_sp = lines_sp[start_sp:start_sp+n_wan]
        start_sp = start_sp+n_wan+2
        coord = lines_uw[i+2:i+2+n_atom]
        count_1 = 0
        count_2 = 0
        start = 0
        start_wan = i+2+n_atom
        print('Calculating {0} frame'.format(count_fr))
        for j in range(len(coord)):
            s1 = coord[j].split()
            
            if s1[0] == 'La':
                wan_1 = float(temp_sp[start].split()[2])
                wan_2 = float(temp_sp[start+1].split()[2])
                wan_3 = float(temp_sp[start+2].split()[2])
                wan_4 = float(temp_sp[start+3].split()[2])
                start = start + 4
                dist = calc_dist(coord[j],lines_uw[start_wan:start_wan+4])
                start_wan = start_wan + 4
                
                wan_frag_1[count_1][:] = wan_1,wan_2,wan_3,wan_4
                wan_frag_1_dist[count_1][:] = dist[0],dist[1],dist[2],dist[3]
                wan_frag_2[count_1][:] = wan_1,wan_2,wan_3,wan_4
                wan_frag_2_dist[count_1][:] = dist[0],dist[1],dist[2],dist[3]
                count_1 = count_1 + 1
            else:
                start = start + 4
                start_wan = start_wan + 4
        #wan_frag_1 = wan_sp[0][:][:]
        #wan_frag_2 = wan_sp[1][:][:]
        #wan_frag_1_dist = wan_dist[0][:][:]
        #wan_frag_2_dist = wan_dist[1][:][:]
        #print(wan_frag_1_dist)
        #print(coord[-1])
        temp_C6,temp_C8=calc_coeff(wan_frag_1,wan_frag_2,wan_frag_1_dist,wan_frag_2_dist)
        C6_final.extend(temp_C6)
        C8_final.extend(temp_C8)

Calculating 1 frame
Calculating 2 frame
Calculating 3 frame
Calculating 4 frame
Calculating 5 frame
Calculating 6 frame
Calculating 7 frame
Calculating 8 frame
Calculating 9 frame
Calculating 10 frame
Calculating 11 frame
Calculating 12 frame
Calculating 13 frame
Calculating 14 frame


In [9]:
((2**0.5)*(sum(C6_final)/len(C6_final)))*(0.529117**6)

2.882472551274574

In [10]:
((2**0.5)*(sum(C8_final)/len(C8_final)))*(0.529117**8)

5.814202762064319

In [15]:
len(C8_final),len(C6_final)

(8568, 8568)

In [78]:
55*1*50

2750

In [10]:
((3.9379158395-3.8172510714)**2+(1.0531292503-0.5907411131 )**2+(-6.5607361025+6.3549491453)**2)**0.5

0.5202989981131498

In [135]:
def doubleIntegral(h, k, lx, ux, ly, uy):
 
    # z stores the table
    # ax[] stores the integral wrt y
    # for all x points considered
    z = [[None for i in range(500)]
               for j in range(500)]
    ax = [None] * 500
 
    # Calculating the number of points
    # in x and y integral
    nx = round((ux - lx) / h + 1)
    ny = round((uy - ly) / k + 1)
    print(nx)
 
    # Calculating the values of the table
    for i in range(0, nx):
        for j in range(0, ny):
            z[i][j] = F(lx + i * h,ly + j * k,sp_1,sp_2)
         
    # Calculating the integral value
    # wrt y at each point for x
    for i in range(0, nx):
        ax[i] = 0
        for j in range(0, ny):
             
            if j == 0 or j == ny - 1:
                ax[i] += z[i][j]
            elif j % 2 == 0:
                ax[i] += 2 * z[i][j]
            else:
                ax[i] += 4 * z[i][j]
         
        ax[i] *= (k / 3)
     
    answer = 0
 
    # Calculating the final integral
    # value using the integral
    # obtained in the above step
    for i in range(0, nx):
        if i == 0 or i == nx - 1:
            answer += ax[i]
        elif i % 2 == 0:
            answer += 2 * ax[i]
        else:
            answer += 4 * ax[i]
     
    answer *= (h / 3)
 
    return answer

In [138]:
# lx and ux are upper and lower limit of x integral
    # ly and uy are upper and lower limit of y integral
    # h is the step size for integration wrt x
    # k is the step size for integration wrt y
lx, ux, ly = 0, lim_x, 0
uy, h, k = lim_y, 0.15, 0.15
print(round(doubleIntegral(h, k, lx, ux, ly, uy), 6)*prefac)

22
7.246513499999998


In [12]:
a = np.array([1,2,3])
b = np.array([1,2,3])
if a==b:
    print('True')

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [19]:
a = [1,2,3]
b = a
print(id(a))
c = a.append(3)
print(id(a),id(b))
print(a,b,c)

140475828977344
140475828977344 140475828977344
[1, 2, 3, 3] [1, 2, 3, 3] None
